# NCO mode CFR test

# Test Description
This script will test the functionality of NCO single mode.
The NCO mode internally generated a sine wave with the frequency and phase.
1) In this script test the CFR functionality for multiple sampling rate from 1GHz to 9GHz on step of 1GHz and multiple CFR from 100MHz to 9GHz on a step on 500MHz for all channels of the proteus module.

# Test Equipment
1. Tabor's Proteus 9484M
2. Keysight's Oscilloscope MSO9254A
3. 4 SMA to BNC Cable


# Hardware Connection

    Proteus Module    --->     Oscilloscope
    
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4
   

# Test Procedure
1. Connect all the channels of Proteus module with the respective channel of oscilloscope.
2. Run the script and observe output on scope.

Proteus release- Aurora

In [ ]:
ip_address = '192.90.70.22'
channb =1

In [ ]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

# Instrument connection

In [ ]:
# Connect to instrument
inst = connect(ip_address)
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{0}" after writing binary values'.format(resp))

# Oscillocope connection

In [ ]:
scope_addr2= 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    ## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))

# Test for different sampling rate

In [ ]:

in_phase =0
test_success = True

for channb in np.arange(1,5,1): #loop for all cahnnels
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))
    if channb%2 ==0:
        maxRate = 2600 #max sampling rate for even channels is 2500MHz
        step = 500
    else:
        maxRate = 10000 #max sampling rate for even channels is 9000MHz
        step = 1000        
    #print(channb,maxRate,step) 
    for sampling_rate in np.arange(1000,maxRate,step): #loop for sampling rate
        inst.send_scpi_cmd(':SOUR:FREQ {}e6'.format(sampling_rate))
        freq = inst.send_scpi_query(':SOUR:FREQ?')
        print('***Test begin for channel {0} and sampling rate {1} MHz ***'.format(channb,float(freq)/1e6))
        for cfreq in np.arange(100,9000,500): #loop for cfr
            inst.send_scpi_cmd(':MODE NCO')
            resp = inst.send_scpi_query(':MODE?')
            #print('source mode', resp, 'selected')
            #NCO mode , nco frequency and NCO phase
            inst.send_scpi_cmd(':NCO:MODE SING')
            resp = inst.send_scpi_query(":NCO:MODE?") #To check mode
            #print('MODE:',resp)
            inst.send_scpi_cmd(':NCO:CFR1 {}e6'.format(cfreq))
            cfr_freq = inst.send_scpi_query(":NCO:CFR1?") #To check frequency
            #print("Carrier frequency ",cfr_freq)
            inst.send_scpi_cmd(':NCO:PHAS1 {0}'.format(in_phase))
            phase = inst.send_scpi_query(":NCO:PHAS1?") #To check phase 
            #print("Phase:{}".format(phase))
            inst.send_scpi_cmd(':VOLT 0.5')
            #inst.send_scpi_cmd(':NCO:SIXD1 ON')
            inst.send_scpi_cmd(':OUTP ON')
            scope.write('AUTOscale')
            time.sleep(5)
            scope.write('*OPC')
            scope.write('*CLS;:DISPlay:CGRade:LEVels')
            scope.write(':MEASure:CLEar')
            scope.write(':MEASure:FREQuency CHANnel{}'.format(channb))
            time.sleep(1)
            scope.write(':MEASure:RESults?')
            result = scope.read()
            frequency = float(result.split(',')[2])/1e6
            #print('Frequency : {}MHz'.format(float(frequency)))
            #print("Ocsilloscope frequency should {}MHz".format(cfreq ))
            difference =abs(cfreq -frequency)
            #print('Difference between measured frequency = {}MHz'.format(abs(cfreq -frequency)))
            if difference < (0.1*cfreq):
                print('\t Test PASS for {0}MHz at sampling rate of {1} MHz'.format(cfreq,sampling_rate))
            else:
                print('\t Test FAIL for {0}MHz at sampling rate of {1} MHz'.format(cfreq,sampling_rate))
                test_success = False

if (test_success):
    print('test successed for CFR')
else:
    print('test failed for CFR')

# Disconnect

In [ ]:
disconnect()
scope.close()
print('Disconnected')